In [1]:
import easyocr

from PIL import Image
import pytesseract

import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

import numpy as np
import Levenshtein as lev

from enum import IntEnum

import cv2
import os
import json

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

C:\ProgramData\Anaconda3\envs\ocr\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!copy E:\datasets\MADE\3_graduation\parthplc\archive\data\train.jsonl E:\datasets\MADE\3_graduation\parthplc\archive\data\train_ocr.jsonl
!copy E:\datasets\MADE\3_graduation\parthplc\archive\data\dev.jsonl E:\datasets\MADE\3_graduation\parthplc\archive\data\dev_ocr.jsonl

‘Є®ЇЁа®ў ­® д ©«®ў:         1.
‘Є®ЇЁа®ў ­® д ©«®ў:         1.


In [3]:
data_dir = r'E:\datasets\MADE\3_graduation\parthplc\archive\data\\'

train_path = data_dir + 'train_ocr.jsonl'
dev_path = data_dir + 'dev_ocr.jsonl'

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(dev_path, lines=True)

In [4]:
train_data['path'] = train_data['img'].apply(lambda x: os.path.join(data_dir, x))
train_data

,id,img,label,text,path
0,42953,img/42953.png,0,its their character not their color that matters,E:\datasets\MADE\3_graduation\parthplc\archive...
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,E:\datasets\MADE\3_graduation\parthplc\archive...
2,13894,img/13894.png,0,putting bows on your pet,E:\datasets\MADE\3_graduation\parthplc\archive...
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,E:\datasets\MADE\3_graduation\parthplc\archive...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",E:\datasets\MADE\3_graduation\parthplc\archive...
...,...,...,...,...,...
8495,10423,img/10423.png,1,nobody wants to hang auschwitz me,E:\datasets\MADE\3_graduation\parthplc\archive...
8496,98203,img/98203.png,1,when god grants you a child after 20 years of ...,E:\datasets\MADE\3_graduation\parthplc\archive...
8497,36947,img/36947.png,1,gays on social media: equality! body positivit...,E:\datasets\MADE\3_graduation\parthplc\archive...
8498,16492,img/16492.png,1,having a bad day? you could be a siamese twin ...,E:\datasets\MADE\3_graduation\parthplc\archive...


In [5]:
test_data['path'] = test_data['img'].apply(lambda x: os.path.join(data_dir, x))
test_data

,id,img,label,text,path
0,8291,img/08291.png,1,white people is this a shooting range,E:\datasets\MADE\3_graduation\parthplc\archive...
1,46971,img/46971.png,1,bravery at its finest,E:\datasets\MADE\3_graduation\parthplc\archive...
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,E:\datasets\MADE\3_graduation\parthplc\archive...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,E:\datasets\MADE\3_graduation\parthplc\archive...
4,80243,img/80243.png,1,mississippi wind chime,E:\datasets\MADE\3_graduation\parthplc\archive...
...,...,...,...,...,...
495,83675,img/83675.png,0,i'm gonna be like phelps one day,E:\datasets\MADE\3_graduation\parthplc\archive...
496,37198,img/37198.png,0,when you're so relaxed you can feel yourself g...,E:\datasets\MADE\3_graduation\parthplc\archive...
497,48670,img/48670.png,0,look at this sandwich maker club i found on wi...,E:\datasets\MADE\3_graduation\parthplc\archive...
498,9863,img/09863.png,0,diverse group of women,E:\datasets\MADE\3_graduation\parthplc\archive...


In [8]:
class ImageTransformType(IntEnum):
    No = 0
    Gray = 1
    Binary = 2
    BinaryInv = 3
    AdaptiveMean = 4
    AdaptiveMeanInv = 5
    AdaptiveGaussian = 6
    AdaptiveGaussianInv = 7

In [9]:
def recognize(path:list, recognizer_name: str, image_transform: ImageTransformType, thresh: int, maxval: int, 
              block_size:int, c:float) -> list:
    recognized_text = []    
    for i in range(len(path)):
        #print(path[i])
        image = cv2.imread(path[i], cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        if image_transform == ImageTransformType.Gray:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
        if image_transform == ImageTransformType.Binary:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY)
            
        if image_transform == ImageTransformType.BinaryInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY_INV)

        if image_transform == ImageTransformType.AdaptiveMean:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)

        if image_transform == ImageTransformType.AdaptiveMeanInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussian:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussianInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if recognizer_name == 'easyocr':            
            recognized_text.append(' '.join(easyocr_reader.readtext(image, detail=0)))
            
        elif recognizer_name == 'pytesseract':
            recognized_text.append(pytesseract.image_to_string(image).replace('\n', ' '))
            
    return recognized_text


def loss(data:list, text:list, recognizer_name: str, image_transform: ImageTransformType, thresh: int, maxval: int, block_size:int, c:float):
    recognized_text = recognize(data, recognizer_name, image_transform, thresh, maxval, block_size, c)
    try:
        distances = []
        for i in range(len(data)):
            txt = text[i]
            distances.append(lev.distance(txt, recognized_text[i]) / len(text))

        distances_mean = np.mean(distances)
    except Exception as ex:
        return 100500
    return distances_mean

In [10]:
loss(test_data['path'].values, 
     test_data['text'].values,
     recognizer_name='pytesseract', 
     image_transform=ImageTransformType.Binary, 
     thresh=253, 
     maxval=158,
     block_size=0,
     c=0)

0.015120000000000001

In [11]:
#Value: 0.015120000000000001
#Parameters: {'image_transform': <ImageTransformType.Binary: 2>, 'thresh': 253, 'maxval': 158}

recognized_text = recognize(test_data['path'].values, 
     recognizer_name='pytesseract', 
     image_transform=ImageTransformType.Binary, 
     thresh=253, 
     maxval=158,
     block_size=0,
     c=0)

dev_rows = pd.read_json(dev_path, lines=True)
with open(data_dir + 'dev_ocr.jsonl', "w") as rf:
    for i, (_, row) in enumerate(dev_rows.iterrows()):
        dictRow = dict(row)
        dictRow["recognized_text_pytesseract"] = recognized_text[i]
        #print(dictRow)
        rf.write(json.dumps(dictRow) + "\n")
        
        
        
recognized_text = recognize(train_data['path'].values, 
     recognizer_name='pytesseract', 
     image_transform=ImageTransformType.Binary, 
     thresh=253, 
     maxval=158,
     block_size=0,
     c=0)


train_rows = pd.read_json(train_path, lines=True)
with open(data_dir + 'train_ocr.jsonl', "w") as rf:
    for i, (_, row) in enumerate(train_rows.iterrows()):
        dictRow = dict(row)
        dictRow["recognized_text_pytesseract"] = recognized_text[i]
        #print(dictRow)
        rf.write(json.dumps(dictRow) + "\n")

In [12]:
easyocr_reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

In [13]:
loss(test_data['path'].values, 
     test_data['text'].values,
     recognizer_name='easyocr', 
     image_transform=ImageTransformType.BinaryInv, 
     thresh=247, 
     maxval=62,
     block_size=0,
     c=0)

0.015208000000000003

In [14]:
#Value: 0.015208000000000003
#Parameters: {'image_transform': <ImageTransformType.BinaryInv: 3>, 'thresh': 247, 'maxval': 62}

recognized_text = recognize(test_data['path'].values, 
     recognizer_name='easyocr', 
     image_transform=ImageTransformType.BinaryInv, 
     thresh=247, 
     maxval=62,
     block_size=0,
     c=0)

dev_rows = pd.read_json(dev_path, lines=True)
with open(data_dir + 'dev_ocr.jsonl', "w") as rf:
    for i, (_, row) in enumerate(dev_rows.iterrows()):
        dictRow = dict(row)
        dictRow["recognized_text_easyocr"] = recognized_text[i]
        #print(dictRow)
        rf.write(json.dumps(dictRow) + "\n")
        
        
        
recognized_text = recognize(train_data['path'].values, 
     recognizer_name='easyocr', 
     image_transform=ImageTransformType.BinaryInv, 
     thresh=253, 
     maxval=95,
     block_size=0,
     c=0)


train_rows = pd.read_json(train_path, lines=True)
with open(data_dir + 'train_ocr.jsonl', "w") as rf:
    for i, (_, row) in enumerate(train_rows.iterrows()):
        dictRow = dict(row)
        dictRow["recognized_text_easyocr"] = recognized_text[i]
        #print(dictRow)
        rf.write(json.dumps(dictRow) + "\n")

In [15]:
train_path = data_dir + 'train_ocr.jsonl'
dev_path = data_dir + 'dev_ocr.jsonl'

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(dev_path, lines=True)

In [16]:
train_data

,id,img,label,text,recognized_text_pytesseract,recognized_text_easyocr
0,42953,img/42953.png,0,its their character not their color that matters,its their character hot their color that matt...,its their character] uot their color that mnat...
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,don't he afraid to love r again everyone is ...,tlon't he afraid t0 love again evervone is not...
2,13894,img/13894.png,0,putting bows on your pet,putting hows on your pet,pvutting hows On1 our pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,iMlovereverything and Bare ios except for squ...,ilove everything and evelbbodlyl except for Sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...","""everyhotly loves chocolate chip cookies, eve...","""everyhody loves chocolate chip cookies, even ..."
...,...,...,...,...,...,...
8495,10423,img/10423.png,1,nobody wants to hang auschwitz me,nobody wants to hang auschwitz me,nohody wants t0 hang auschwitz mne
8496,98203,img/98203.png,1,when god grants you a child after 20 years of ...,vhien god grants you a child after 20 years of...,whcu god grants VOl @ childl aftcr 20 vcars of...
8497,36947,img/36947.png,1,gays on social media: equality! body positivit...,yay cn stivial meta: etucniy! tally positivity...,Walys UIII SIIi%l Mli;l: #IU;Iv! Eglu HUSMUMy!...
8498,16492,img/16492.png,1,having a bad day? you could be a siamese twin ...,having a bad tay? vou couli le a siamese twin ...,having a had Ilay? VOL COULil he a siamese [WI...


In [17]:
test_data

,id,img,label,text,recognized_text_pytesseract,recognized_text_easyocr
0,8291,img/08291.png,1,white people is this a shooting range,white people is this a shooting range,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest,bravery aLilsfinest,hravery ~abils [iuest ;
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,your order comes to $37.50 and your white pri...,vour order comes t0 S37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,itis time.. to send these parasites hack to t...,Jitis Lime. t0 scnd thcsc Qarasitcs hack to th...
4,80243,img/80243.png,1,mississippi wind chime,mississippi wind chime,mississippi wind chime
...,...,...,...,...,...,...
495,83675,img/83675.png,0,i'm gonna be like phelps one day,rm gonna he like phelps one day,0'071 gonna he Iike pvhelps one day
496,37198,img/37198.png,0,when you're so relaxed you can feel yourself g...,when you're so relaxed you can feel yourself ...,when vou're SO relaxed vou can feel yourself g...
497,48670,img/48670.png,0,look at this sandwich maker club i found on wi...,look at this sandwich inaker club Hfound on w...,look at this sandwich maker club fount 0n wish...
498,9863,img/09863.png,0,diverse group of women,CIUEFSEWQKOUD Of women,diverse @eouj) 0f [omen
